<a href="https://colab.research.google.com/github/stiflerGit/CAD-Mammography/blob/master/ENSEMBLE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
 # CONSTANTS
    
MODELS = [
    {
        'name': 'ScratchCNN', 
        'version': 'v0.0.01', 
        'dir':'/content/drive/My Drive/Scratch_CNN/'
    },
#     {
#         'name': 'Pretrained_CNN',
#         'version':'v0.0.02',
#         'dir': '/content/drive/My Drive/PreTrained_CNN/'
#     },
    {
        'name': 'Pretrained-intermediate', 
        'version': 'v0.0.01', 
        'dir':'/content/drive/My Drive/PreTrained_CNN-Intermediate/'
    },
    {
        'name': 'ResnetV2', 
        'version': 'v0.0.01', 
        'dir':'/content/drive/My Drive/ResNet/'
    },
]

WORK_DIR = './drive/My Drive/ENSEMBLE/'

VERSION = 'v0.0.01'

# dataset
IMAGE_SIZE=[224,224]
DATASET_PATH  = '/content/drive/My Drive/CI_FinalProject/'
TRAIN_IMGS_FILE_NAME = 'train_img_%d.npy' % IMAGE_SIZE[0]
TRAIN_LABELS_FILE_NAME = 'train_lab.npy'
TEST_IMGS_FILE_NAME = 'public_test_image_%d.npy' % IMAGE_SIZE[0]
TEST_LABELS_FILE_NAME = 'public_test_label.npy'
BATCH_SIZE=64
val_split=0.2


In [2]:
import numpy as np
from google.colab import drive
import os
drive.mount('/content/drive')

if not os.path.exists(WORK_DIR):
    os.makedirs(WORK_DIR)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


We have to preprocess data in the same way we did in the training

In [3]:
# PRE-PROCESS DATASET

import numpy as np
from sklearn.model_selection import train_test_split

train_gray_imgs = np.load(os.path.join(DATASET_PATH, TRAIN_IMGS_FILE_NAME))
train_lbls = np.load(os.path.join(DATASET_PATH, TRAIN_LABELS_FILE_NAME))

print(train_lbls.shape) # 2864 images (it is a quite small dataset)
print(np.count_nonzero(train_lbls)) # 1546 
# 1546 images belong to class 1, this means 1318 images belong to class 0
# so our class distribution is good

# we have to equally distribute otherwise the error:
### Training and validation subsets have different number of classes after the 
### split. If your numpy arrays are sorted by the label, you might want to shuffle them.
# could be raised. This is due to the distribution of classes of the labels.
# for example after the split, the training labels set contains both class 1 and 2,
# and on the other hand validation labels set contains either class 1 or class 2.
# Even if the train data would cointain both the classes it's better to equally 
# distribute the two classes over the sets.
X_train, X_val, y_train, y_val = train_test_split(train_gray_imgs, train_lbls,
                                                  test_size=val_split, 
                                                  stratify=train_lbls)

del(X_train, y_train)
valid_samples = len(X_val)

X_val = 1./32768 * X_val
X_val = X_val - 1.
# this is done to transform the gray imgs into rgb images
X_val = np.stack((X_val,)*3, axis=-1)

del(train_gray_imgs)
del(train_lbls)

test_gray_imgs = np.load(os.path.join(DATASET_PATH, TEST_IMGS_FILE_NAME))
test_gray_imgs = 1./32768 * test_gray_imgs
test_gray_imgs = test_gray_imgs - 1.
test_rgb_imgs = np.stack((test_gray_imgs,)*3, axis=-1)
test_lbls = np.load(os.path.join(DATASET_PATH, TEST_LABELS_FILE_NAME))

del(test_gray_imgs)

(2864,)
1546


In [4]:
# PRE-PROCESS DATASET 

from keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator()
validation_iterator = test_datagen.flow(X_val, y_val, batch_size=BATCH_SIZE, shuffle=False)
test_iterator = test_datagen.flow(test_rgb_imgs, test_lbls, batch_size=BATCH_SIZE)

Using TensorFlow backend.


In [0]:
import json
import h5py
from keras import models

# https://github.com/keras-team/keras/issues/11683

def fix_layer0(filename, batch_input_shape, dtype):
    with h5py.File(filename, 'r+') as f:
        model_config = json.loads(f.attrs['model_config'].decode('utf-8'))
        layer0 = model_config['config']['layers'][0]['config']
        layer0['batch_input_shape'] = batch_input_shape
        layer0['dtype'] = dtype
        f.attrs['model_config'] = json.dumps(model_config).encode('utf-8')
        
def open_model(model_path, input_shape):
    while True:
        try:
            m = models.load_model(model_path)
            return m
        except:
            fix_layer0(model_path, input_shape, 'float32')


In [0]:
import os 
from keras import models as models

input_shape = [None, IMAGE_SIZE[0], IMAGE_SIZE[1], 3]

mymodels = []

for i,m in enumerate(MODELS):
    model_path = os.path.join(m['dir'], m['version'], 'model') + '.h5'
    print('loading model at path %s'%model_path)
    mymodels.append(open_model(model_path, input_shape))
    print('model loaded')
    

loading model at path /content/drive/My Drive/Scratch_CNN/v0.0.01/model.h5
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
model loaded
loading model at path /content/drive/My Drive/PreTrained_CNN-Intermediate/v0.0.01/model.h5
model loaded
loading model at path /content/drive/My Drive/ResNet/v0.0.01/model.h5


In [0]:
from keras import Input
from keras.models import Model
from keras.layers import Average

input_tensor = Input(shape=(224,224,3), name='input_tensor', dtype='float32')

output_tensors = []
for m in mymodels:
    output_tensor = m(input_tensor)
    output_tensors.append(output_tensor)
    
output_tensor = Average()(output_tensors)
ensemble_model = models.Model(input_tensor, output_tensor, name='ensemble')

ensemble_model.summary()

predicts = ensemble_model.predict(X_val) 
predicts = predicts.argmax(axis=-1)
print(predicts)

# def ensemble(models, model_input):
    
#     outputs = [model.outputs[0] for model in models]
#     y = Average()(outputs)
    
#     model = Model(model_input, y, name='ensemble')
    
#     return model

# input_tensor = Input(shape=(224,224,3), name='input_tensor', dtype='float32')
# ensemble_model = ensemble(mymodels,input_tensor)



In [0]:
# evaluate

def evaluate(predicted, labels):
    correct = 0
    for i in range(len(labels)):
        if predicted[i] == labels[i]:
            correct = correct+1
    return correct / len(labels)

for i,m in enumerate(mymodels):
    categorical_predicts = m.predict(X_val)
    predicts = np.argmax(categorical_predicts, axis=-1)
    print('model_{} evaluate: {}'.format(MODELS[i]["name"],
                                        evaluate(predicts, y_val)))
categorical_predicts = ensemble_model.predict(X_val)
predicts = np.argmax(categorical_predicts, axis=-1)
print('ensemble evaluate: {}'.format(evaluate(predicts, y_val)))

In [0]:
# for i,m in enumerate(mymodels):
#     m.evaluate(X,y)
    
# ensemble_model.evaluate(X,y)